In [1]:
import sys, os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

from RFSoC_Daq import RFSoC_Daq
from Biquad import Biquad
from SimBiquad import SimBiquad
from Biquad_Daq import Biquad_Daq
from Filter import Filter


In [2]:
daq = Biquad_Daq()
sim = SimBiquad()
filter = Filter()

filter.calc_params(460)

DEBUG:RFSoC_Daq:Asked to load overlay at /home/xilinx/python/zcumts.py
DEBUG:RFSoC_Daq:Adding directory /home/xilinx/python to module search path
DEBUG:RFSoC_Daq:Changing directory to /home/xilinx/python
DEBUG:RFSoC_Daq:Going to try to import zcumts
DEBUG:RFSoC_Daq:Found Overlay class Overlay from module pynq.overlay
DEBUG:RFSoC_Daq:Found RFSoC overlay zcuMTS


Turning on SYNC
Turning off SYNC


DEBUG:RFSoC_Daq:Created RFSoC device
DEBUG:RFSoC_Daq:Restoring original module search path
DEBUG:RFSoC_Daq:Going back to original directory /home/xilinx/rfsoc-pydaq/Biquad/Notebooks


In [3]:
import pynq

import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual, HBox
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

In [4]:
def update_plot():
    # daq.reset()
    
    daq.update_params(**filter.get_params())
    daq.quantise_coeffs()
    
    daq.generate_waveforms()
    
    sim.update_params(**filter.get_params())
    sim.data = daq.adcBuffer[0] >> 4
    sim.quantise_coeffs()
    
    sim.run_IIR()
    
    daq_output = daq.extract_biquad()
    sim_output = sim.extract_biquad()
    daq_input = daq.extract_raw()
    
    fig, ((axB,axS,axR),(ax0, ax1, ax2)) = plt.subplots(2, 3, figsize=(35, 15))
        
    daq_output.plotWaveform(axB, title="Biquaded ADC224_T0_CH0")
    daq_output.plotFFT(ax0, title="Biquaded FFT")
    
    sim_output.plotWaveform(axS, title="Simulated Biquad")
    sim_output.plotFFT(ax1, title="Simulated FFT")
    
    daq_input.plotWaveform(axR, title="Raw ADC224_T0_CH0")
    daq_input.plotFFT(ax2, title="Raw FFT")
    
    plt.show()
    print(filter)

In [5]:
A_slider = widgets.FloatSlider(min=-4, max=4, step=0.05, value=filter.A, description='A:', continuous_update=False)
P_slider = widgets.FloatSlider(min=0, max=1, step=0.05, value=filter.P, description='P:', continuous_update=False)
Zero_slider = widgets.FloatSlider(min=200, max=1200, step=5, value=filter.fz, description='Zero:', continuous_update=False)
Pole_slider = widgets.FloatSlider(min=1, max=1500, step=5, value=filter.fp, description='Pole:', continuous_update=False)

plot_output = widgets.Output()

# def update_plot():
#     with plot_output:
#         plot_output.clear_output(wait=True)
#        print(f"Plotting with: A={filter.A}, P={filter.P}, Zero={filter.fz}, Pole={filter.fp}")

def update_params(_=None):
    with plot_output:
        plot_output.clear_output(wait=True)
        filter.fz = Zero_slider.value
        filter.A = A_slider.value
        filter.P = P_slider.value
        filter.fp = Pole_slider.value
        update_plot()

def update(change):
    with plot_output:
        plot_output.clear_output(wait=True)
        if change['owner'] == Zero_slider:
            filter.fz = Zero_slider.value
            A_slider.value = filter.A
            P_slider.value = filter.P
            Pole_slider.value = filter.fp
        update_plot()
        
update_button = widgets.Button(description="Update Plot")

update_button.on_click(update_params)
        
Zero_slider.observe(update, names='value')

# Layout
slider_box = widgets.HBox([A_slider, P_slider, Zero_slider, Pole_slider, update_button])
ui = widgets.VBox([slider_box, plot_output])

# Display
display(ui)

# Initial plot
update_params()